In [1]:
import pdfplumber
import re
import pandas as pd
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

In [2]:
path = '/Users/asen/Desktop/finance_files/'
filenames = os.listdir(f'{path}')
filenames = [path + i for i in filenames]
filenames = sorted(filenames)
filenames = filenames[1:] #delete ds store
filenames =  sorted(filenames)
filenames

['/Users/asen/Desktop/finance_files/jul2022_sep2022_1.pdf',
 '/Users/asen/Desktop/finance_files/jul2022_sep2022_3.pdf',
 '/Users/asen/Desktop/finance_files/jul2022_sep2022_4.pdf',
 '/Users/asen/Desktop/finance_files/jul2022_sep2022_5.pdf',
 '/Users/asen/Desktop/finance_files/jul2022_sep2022_6.pdf',
 '/Users/asen/Desktop/finance_files/jul2022_sep2022_amendment_affidavit_2.pdf']

In [3]:
names = {'/Users/asen/Desktop/finance_files/jul2022_sep2022_3.pdf': 'Clay Jenkins',
 '/Users/asen/Desktop/finance_files/jul2022_sep2022_4.pdf': 'John Creuzot',
 '/Users/asen/Desktop/finance_files/jul2022_sep2022_5.pdf': 'Faith Johnson',
 '/Users/asen/Desktop/finance_files/jul2022_sep2022_6.pdf': 'Andrew Sommerman', 
 '/Users/asen/Desktop/finance_files/jul2022_sep2022_1.pdf': 'Justin Koch',
 '/Users/asen/Desktop/finance_files/jul2022_sep2022_amendment_affidavit_2.pdf': 'Lauren Davis'
 }

In [4]:
df = []

fps = ['Date Full', 'City State', 'Zip Code', 'Date Payee', 'Zip  Code', 'Ave,  Suite', 'Date  Full', 'City,  State', 'State,  Zip', 'Date  Fullname', 'Zip Cod', 'See  Instructions', 'Date  Payee', 'See  Instructons']

for file in filenames:
    # os.system(f'ocrmypdf --force-ocr --deskew --clean-final {file} {file}')

    text = ''
    with pdfplumber.open(f"{file}") as pdf:
        for i in range(3, len(pdf.pages)):
            page = pdf.pages[i]
            text += page.extract_text()
    
    lines  = text.split('\n')

    lines = [i for i in lines if i.strip() != '']

    date_idx = []

    for idx, line in enumerate(lines):
        if 'Date' in line:
            date_idx.append(idx) 
        else:
            pass
    
    

    for i, j in enumerate(date_idx):

        d = {}

        if i != 0: 
            chunk = lines[(date_idx[i-1]):(date_idx[i])]

            chunk = [i for i in chunk if i.strip() != '']
            
            chunk_text = ''
            
            for line in chunk:
                chunk_text = chunk_text + '\n' + line
            
            chunk_text = chunk_text.replace(' /', '/').replace('/ ','/')

            #get candidate name
            d['candidate_name'] = names[file]

            #get candidate date
            if re.findall('\d{1,2}/\d{1,2}/\d{2,4}', chunk_text) != []:
                d['date'] = re.findall('\d{1,2}/\d{1,2}/\d{2,4}', chunk_text, re.MULTILINE)[0]
                
            elif re.findall('\d{1,2}\-\d{1,2}\-\d{2,4}', chunk_text) != []:
                d['date'] = re.findall('\d{1,2}\-\d{1,2}\-\d{2,4}', chunk_text, re.MULTILINE)[0]

            elif re.findall('\d+/\d+\s\d+', chunk_text) != []:
                d['date'] = re.findall('\d+/\d+\s\d+', chunk_text, re.MULTILINE)[0].replace(' ', '') + ('/22')

            else:
                d['date'] = ''
            
            #get amount contributed
            if re.findall('\$\d+\.\d{2}', chunk_text) != []:
                d['amount'] = re.findall('\$\d+\.\d{2}', chunk_text)[0]
            
            elif re.findall('\$\d+', chunk_text) != []:
                d['amount'] = re.findall('\$\d+', chunk_text)[0]
            
            elif re.findall('\$\d{1,3}(,\d{3})*(\.\d{2})?', chunk_text) != []:
                d['amount'] = re.findall('\$\d{1,3}(,\d{3})*(\.\d{2})?', chunk_text)[0]

            elif re.findall('\d+\.\d{2}', chunk_text) != []:
                d['amount'] = re.findall('\d+\.\d{2}', chunk_text)[0]
            
            elif re.findall('\d{1,3}(,\d{3})*(\.\d{2})?', chunk_text) != []:
                d['amount'] = re.findall('\d{1,3}(,\d{3})*(\.\d{2})?', chunk_text)[0]

            else:
                d['amount'] = ''


            #get contributor name
            if d['candidate_name'] == 'Justin Koch' or d['candidate_name'] == 'Lauren Davis':
                d['contributor_name'] = chunk[1].replace(d['date'], '').replace(str(d['amount']), '')
            
            elif re.findall("[A-Z][a-z]+,  [A-Z][a-z]+", chunk_text) != []: #comma space space
                counter = 0
                while counter < len(re.findall("[A-Z][a-z]+,  [A-Z][a-z]+", chunk_text)):
                    if re.findall("[A-Z][a-z]+,  [A-Z][a-z]+", chunk_text)[counter] not in fps and 'Dallas' not in re.findall("[A-Z][a-z]+,  [A-Z][a-z]+", chunk_text)[counter]:
                        d['contributor_name'] = re.findall("[A-Z][a-z]+,  [A-Z][a-z]+", chunk_text)[counter]
                        break
                    else:
                        counter += 1

            elif re.findall("[A-Z][a-z]+  [A-Z][a-z]+", chunk_text) != []: #space space
                counter = 0
                while counter < len(re.findall("[A-Z][a-z]+  [A-Z][a-z]+", chunk_text)):
                    if re.findall("[A-Z][a-z]+  [A-Z][a-z]+", chunk_text)[counter] not in fps and 'Dallas' not in re.findall("[A-Z][a-z]+  [A-Z][a-z]+", chunk_text)[counter]:
                        d['contributor_name'] = re.findall("[A-Z][a-z]+  [A-Z][a-z]+", chunk_text)[counter]
                        break
                    else:
                        counter += 1
            
            elif re.findall("[A-Z][a-z]+ [A-Z][a-z]+", chunk_text) != []: #space
                counter = 0
                while counter < len(re.findall("[A-Z][a-z]+ [A-Z][a-z]+", chunk_text)):
                    if re.findall("[A-Z][a-z]+ [A-Z][a-z]+", chunk_text)[counter] not in fps and 'Dallas' not in re.findall("[A-Z][a-z]+ [A-Z][a-z]+", chunk_text)[counter]:
                        d['contributor_name'] = re.findall("[A-Z][a-z]+ [A-Z][a-z]+", chunk_text)[0]
                        break
                    else:
                        counter += 1
            
            else:
                d['contributor_name'] = ''
            
            
            d['text'] = chunk_text

            df.append(d)
    
    else:
        pass

df = pd.DataFrame(df)


#lauren davis received x amount of $5k or more 
#how many contributions? 
#x amount, with x amount over $xk dollars 
    


In [5]:
print(len(df.loc[df['amount'] != '']) / len(df))
print(len(df.loc[df['date'] != '']) / len(df))
print(len(df.loc[df['contributor_name'] != '']) / len(df))

0.9994435169727324
0.9337785197551475
1.0


In [6]:
df['amount'] = df['amount'].str.strip()
df['amount'] = df['amount'].str.replace('$', '').replace(',', '')
# df['amount'] = pd.to_numeric(df['amount'], errors='coerce')
# df['date'] = pd.to_datetime(df['date'], errors='coerce')

/var/folders/kd/czy8636s60z_vkm36s066th80000gq/T/ipykernel_83651/3134671947.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['amount'] = df['amount'].str.replace('$', '').replace(',', '')


In [7]:
df.to_csv('judges_campiagn_contributions5.csv', index=False)